In [ ]:
#pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

# REDES RECURRENTES LSTM

In [ ]:
# Datos de entrada (secuencia de meses)
secuencia_cliente_1 = np.array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0])
secuencia_cliente_2 = np.array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0])

# Datos de salida (representación de seguros actuales)
seguros_cliente_1 = np.array([1, 1, 0, 0, 1])  # Ejemplo: El cliente 1 tiene seguros 1, 2, y 5
seguros_cliente_2 = np.array([1, 0, 0, 0, 0])  # Ejemplo: El cliente 2 solo tiene seguro 1

# Construye el modelo de RNN
model = Sequential()
model.add(LSTM(32, input_shape=(12, 1)))  # 12 meses como entrada, 1 unidad de tiempo
model.add(Dense(5, activation='sigmoid'))  # 5 seguros como salida

# Compila el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrena el modelo con los datos de ejemplo
X_train = np.array([secuencia_cliente_1, secuencia_cliente_2])
X_train = np.expand_dims(X_train, axis=-1)  # Agrega una dimensión para la entrada de tiempo
y_train = np.array([seguros_cliente_1, seguros_cliente_2])

model.fit(X_train, y_train, epochs=100, batch_size=2)

# Genera una recomendación para un nuevo cliente (simulado)
nuevo_cliente = np.array([0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0])
nuevo_cliente = np.expand_dims(nuevo_cliente, axis=0)

prediccion = model.predict(nuevo_cliente)

num_recomendaciones = 3  # Puedes ajustar este número según tus preferencias
recomendaciones = np.argsort(prediccion[0])[::-1][:num_recomendaciones]

print("Recomendación de seguros para el nuevo cliente:", recomendaciones)

In [ ]:
# Generar datos de ejemplo (simulados)
np.random.seed(0)
n_samples = 1000
n_timesteps = 12  # 12 meses de datos

# Características del cliente: Edad, Ingresos, Localidad (1-5), Tipo de Industria (1-3)
X_client = np.random.uniform(18, 65, size=(n_samples, 4))
X_income = np.random.uniform(20000, 100000, size=(n_samples, 1))
X_location = np.random.randint(1, 6, size=(n_samples, 1))
X_industry = np.random.randint(1, 4, size=(n_samples, 1))

# ID del cliente (identificador único)
client_ids = np.arange(1, n_samples + 1).reshape(-1, 1)

# Series temporales de coberturas adquiridas (simuladas como valores binarios)
X_coverages = np.random.randint(2, size=(n_samples, n_timesteps, 3))

# Series temporales de contactos del cliente (simuladas como valores binarios)
X_contacts = np.random.randint(2, size=(n_samples, n_timesteps, 1))

# Características de riesgo por desastres naturales y seguridad por robo por provincia (simuladas)
X_risk_features = np.random.rand(n_samples, n_timesteps, 2)
X_security_features = np.random.rand(n_samples, n_timesteps, 2)

# Objetivo: Recomendación de seguros (etiquetas codificadas en one-hot)
y = np.random.randint(2, size=(n_samples, n_timesteps, 3))

# Dividir el conjunto de datos en entrenamiento y prueba
split_ratio = 0.8
split_index = int(n_samples * split_ratio)

X_client_train = X_client[:split_index]
X_client_test = X_client[split_index:]
client_ids_train = client_ids[:split_index]
client_ids_test = client_ids[split_index:]
X_coverages_train = X_coverages[:split_index]
X_coverages_test = X_coverages[split_index:]
X_contacts_train = X_contacts[:split_index]
X_contacts_test = X_contacts[split_index:]
X_risk_train = X_risk_features[:split_index]
X_risk_test = X_risk_features[split_index:]
X_security_train = X_security_features[:split_index]
X_security_test = X_security_features[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

# Definir la arquitectura del modelo
input_client = Input(shape=(4,))
input_coverages = Input(shape=(n_timesteps, 3))
input_contacts = Input(shape=(n_timesteps, 1))
input_risk = Input(shape=(n_timesteps, 2))
input_security = Input(shape=(n_timesteps, 2))
input_id = Input(shape=(1,))

In [ ]:
# Capa de embeddings para el ID del cliente
embedding_id = Embedding(input_dim=n_samples + 1, output_dim=32)(input_id)
embedding_id = tf.reduce_mean(embedding_id, axis=1)

# Capa de embeddings para características del cliente
embedding_client = Dense(32, activation='relu')(input_client)

# Capa LSTM para series temporales de coberturas adquiridas y contactos del cliente
lstm_output = LSTM(64, return_sequences=True)(input_coverages)
lstm_output = Concatenate(axis=-1)([lstm_output, input_contacts])

# Capa densa para características de riesgo y seguridad
dense_risk = Dense(16, activation='relu')(input_risk)
dense_security = Dense(16, activation='relu')(input_security)


In [ ]:
# # Combinar todas las características
# combined_features = Concatenate()([embedding_id, embedding_client, lstm_output, dense_risk, dense_security])

In [ ]:
# # Capa de salida para clasificación de seguros
# output = Dense(3, activation='softmax')(combined_features)

# # Construir el modelo
# model = Model(inputs=[input_client, input_coverages, input_contacts, input_risk, input_security, input_id], outputs=output)

# # Compilar el modelo
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Entrenar el modelo
# model.fit([X_client_train, X_coverages_train, X_contacts_train, X_risk_train, X_security_train, client_ids_train],
#           y_train,
#           epochs=10,
#           batch_size=32,
#           validation_data=([X_client_test, X_coverages_test, X_contacts_test, X_risk_test, X_security_test, client_ids_test], y_test))

# # Hacer predicciones (por ejemplo, para un nuevo cliente y mes)
# new_client_id = np.array([[n_samples + 1]])  # ID de cliente nuevo
# new_client_features = np.array([[35, 60000, 3, 2]])  # Ejemplo de características de cliente nuevo
# new_coverages = np.random.randint(2, size=(1, n_timesteps, 3))  # Ejemplo de series temporales de coberturas
# new_contacts = np.random.randint(2, size=(1, n_timesteps, 1))  # Ejemplo de series temporales de contactos
# new_risk = np.random.rand(1, n_timesteps, 2)  # Ejemplo de características de riesgo
# new_security = np.random.rand
